In [156]:
from dataclasses import dataclass
import numpy as np
from typing import List
from ortools.linear_solver import pywraplp

In [157]:
def random_graph(n: int, p: float = 0.25):
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            if np.random.rand() < p:
                matrix[i, j] = 1
                matrix[j, i] = 1
    return matrix


@dataclass
class Agent:
    licenses: List[int]
    bids: List[List[float]]
    crossBids: List[List[float]]


def generate(regions: int = 12, #упрощенная версия генерации, чтоб не тянуть время
             blocks: int = 16, 
             agents: int = 20,
             T: float = 1.5):
    matrix = random_graph(regions)
    remains = [blocks] * regions
    A = []
    for agent in range(agents):
        licenses = [0] * regions
        for region in range(regions):
            n = remains[region]
            probabilities = np.exp(-np.arange(n + 1) / T)
            probabilities /= np.sum(probabilities)
            l = np.random.choice(np.arange(n + 1), p=probabilities)
            remains[region] -= l
            licenses[region] = l
        bids = [[]] * regions
        for region in range(regions):
            l = licenses[region]
            if l != 0:
                s = blocks + 1 - l
                bids[region] = (np.random.rand(s) <= 0.6) * np.random.rand(s) * 10 ** 6 #ставки на частоты
        crossBids = np.zeros((regions, regions))
        for region in range(regions):
            for region_ in range(region + 1, regions):
                if matrix[region, region_] != 0 and licenses[region] != 0 and licenses[region_] != 0:
                    D = np.random.rand() * 10 ** 3 # размер ставки на пересечение
                    crossBids[region, region_] = D
                    crossBids[region_, region] = D
        A.append(Agent(licenses, bids, crossBids))
    return A

In [158]:
regions, blocks, agents = 12, 16, 20
A = generate(regions, blocks, agents)
sorted([sum(a.licenses) for a in A]) # семплим, пока не будет красиво =)

[1, 2, 5, 7, 7, 7, 8, 8, 9, 9, 9, 10, 11, 12, 12, 13, 14, 15, 16, 17]

In [162]:
def AssignmentRound(regions: int, blocks: int, A: List[Agent]):
    agents = len(A)
    solver = pywraplp.Solver.CreateSolver("SAT")
    # тут хранится выбор блоков 
    x = [[[solver.BoolVar(name = "x_{:d}_{:d}_{:d}".format(agent, region, block)) 
            for block in range(blocks + 1 - A[agent].licenses[region])] 
            for region in range(regions)] 
            for agent in range(agents)] 
    # для каждого агента в каждом регионе должен быть выбран блок
    for agent in range(agents): 
        for region in range(regions):
            if len(x[agent][region]) > 0:
                solver.Add(sum(x[agent][region]) == 1)
    # пересекающиеся блоки нельзя взять 
    for region in range(regions):
        for freq in range(blocks): # проблемы с именами, тут это частоты
            solver.Add(
                sum([sum([x[agent][region][i]
                for i in range(len(x[agent][region])) 
                if i <= freq and freq <= i + A[agent].licenses[region] - 1]) 
                for agent in range(agents)]) <= 1)
    # переменные с пересечением
    y = [[[[solver.BoolVar(name = "y_{:d}_{:d}_{:d}_{:d}".format(agent, region, region_, freq)) 
            for freq in range(blocks)] 
            for region_ in range(regions)] 
            for region in range(regions)] 
            for agent in range(agents)]
    
    # пересечение <=> частота взята там И там
    for agent in range(agents):
        for region in range(regions):
            for region_ in range(regions):
                for freq in range(blocks):
                    solver.Add(y[agent][region][region_][freq] <= 0.5 * (
                        sum([x[agent][region][i] for i in range(len(x[agent][region])) 
                            if i <= freq and freq <= i + A[agent].licenses[region] - 1]) +
                        sum([x[agent][region_][i] for i in range(len(x[agent][region_])) 
                            if i <= freq and freq <= i + A[agent].licenses[region_] - 1])))


    
    return 0

AssignmentRound(regions, blocks, A)

0